In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 2
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.84 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_5S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torchvision\ops\misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(


Number of classes defined: 50, but try to load pre-trained layer with logits: 527
Dropping last layer.
Error(s) in loading state_dict for MobileNetV3:
	Missing key(s) in state_dict: "classifier.5.weight", "classifier.5.bias". 
Loading weights pre-trained weights in a non-strict manner.
MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:867.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


Train Epoch: 1/50 Train_Loss: 3.9241464138031006 Train_acc: 2.437500%
Train Epoch: 2/50 Train_Loss: 3.934868574142456 Train_acc: 2.562500%
Train Epoch: 3/50 Train_Loss: 3.906146287918091 Train_acc: 2.125000%
Train Epoch: 4/50 Train_Loss: 3.8779680728912354 Train_acc: 2.312500%
Train Epoch: 5/50 Train_Loss: 3.8697993755340576 Train_acc: 3.812500%
Train Epoch: 6/50 Train_Loss: 3.835264205932617 Train_acc: 4.937500%
Train Epoch: 7/50 Train_Loss: 3.7132623195648193 Train_acc: 10.312500%
Train Epoch: 8/50 Train_Loss: 3.522891044616699 Train_acc: 21.812500%
Train Epoch: 9/50 Train_Loss: 3.4158122539520264 Train_acc: 42.437500%
Train Epoch: 10/50 Train_Loss: 3.160552740097046 Train_acc: 64.625000%
Train Epoch: 11/50 Train_Loss: 2.7389442920684814 Train_acc: 79.375000%
Train Epoch: 12/50 Train_Loss: 2.378939628601074 Train_acc: 88.000000%
Train Epoch: 13/50 Train_Loss: 2.0805561542510986 Train_acc: 90.250000%
Train Epoch: 14/50 Train_Loss: 1.5033838748931885 Train_acc: 92.750000%
Train Epoch: 

Train Epoch: 1/50 Train_Loss: 3.9828414916992188 Train_acc: 1.250000%
Train Epoch: 2/50 Train_Loss: 3.9590892791748047 Train_acc: 1.562500%
Train Epoch: 3/50 Train_Loss: 3.9445929527282715 Train_acc: 1.187500%
Train Epoch: 4/50 Train_Loss: 3.9572770595550537 Train_acc: 1.562500%
Train Epoch: 5/50 Train_Loss: 3.9220926761627197 Train_acc: 1.812500%
Train Epoch: 6/50 Train_Loss: 3.830979585647583 Train_acc: 2.750000%
Train Epoch: 7/50 Train_Loss: 3.721797466278076 Train_acc: 6.687500%
Train Epoch: 8/50 Train_Loss: 3.605010747909546 Train_acc: 17.750000%
Train Epoch: 9/50 Train_Loss: 3.441118001937866 Train_acc: 41.250000%
Train Epoch: 10/50 Train_Loss: 3.1413183212280273 Train_acc: 62.375000%
Train Epoch: 11/50 Train_Loss: 2.7967159748077393 Train_acc: 78.750000%
Train Epoch: 12/50 Train_Loss: 2.5648465156555176 Train_acc: 86.312500%
Train Epoch: 13/50 Train_Loss: 1.9293988943099976 Train_acc: 90.125000%
Train Epoch: 14/50 Train_Loss: 1.6096614599227905 Train_acc: 92.312500%
Train Epoch:

Train Epoch: 1/50 Train_Loss: 3.9326694011688232 Train_acc: 1.500000%
Train Epoch: 2/50 Train_Loss: 3.9654147624969482 Train_acc: 1.187500%
Train Epoch: 3/50 Train_Loss: 3.9353365898132324 Train_acc: 2.187500%
Train Epoch: 4/50 Train_Loss: 3.9185385704040527 Train_acc: 2.437500%
Train Epoch: 5/50 Train_Loss: 3.884451389312744 Train_acc: 2.875000%
Train Epoch: 6/50 Train_Loss: 3.8043222427368164 Train_acc: 4.437500%
Train Epoch: 7/50 Train_Loss: 3.7449119091033936 Train_acc: 8.937500%
Train Epoch: 8/50 Train_Loss: 3.6027333736419678 Train_acc: 21.125000%
Train Epoch: 9/50 Train_Loss: 3.4426374435424805 Train_acc: 41.500000%
Train Epoch: 10/50 Train_Loss: 3.200963258743286 Train_acc: 64.687500%
Train Epoch: 11/50 Train_Loss: 2.6880648136138916 Train_acc: 79.812500%
Train Epoch: 12/50 Train_Loss: 2.3717775344848633 Train_acc: 87.312500%
Train Epoch: 13/50 Train_Loss: 1.9293283224105835 Train_acc: 89.500000%
Train Epoch: 14/50 Train_Loss: 1.5161739587783813 Train_acc: 92.062500%
Train Epoc

Train Epoch: 1/50 Train_Loss: 3.9911680221557617 Train_acc: 1.187500%
Train Epoch: 2/50 Train_Loss: 3.975062131881714 Train_acc: 1.250000%
Train Epoch: 3/50 Train_Loss: 3.954847812652588 Train_acc: 1.625000%
Train Epoch: 4/50 Train_Loss: 3.9142508506774902 Train_acc: 1.750000%
Train Epoch: 5/50 Train_Loss: 3.912996292114258 Train_acc: 2.375000%
Train Epoch: 6/50 Train_Loss: 3.850691795349121 Train_acc: 4.562500%
Train Epoch: 7/50 Train_Loss: 3.774817943572998 Train_acc: 9.250000%
Train Epoch: 8/50 Train_Loss: 3.674961805343628 Train_acc: 20.937500%
Train Epoch: 9/50 Train_Loss: 3.409250259399414 Train_acc: 38.312500%
Train Epoch: 10/50 Train_Loss: 3.1650583744049072 Train_acc: 59.687500%
Train Epoch: 11/50 Train_Loss: 2.7750303745269775 Train_acc: 74.750000%
Train Epoch: 12/50 Train_Loss: 2.5615508556365967 Train_acc: 82.125000%
Train Epoch: 13/50 Train_Loss: 1.9687645435333252 Train_acc: 86.500000%
Train Epoch: 14/50 Train_Loss: 1.722663164138794 Train_acc: 89.375000%
Train Epoch: 15/

Train Epoch: 1/50 Train_Loss: 3.925394296646118 Train_acc: 2.687500%
Train Epoch: 2/50 Train_Loss: 3.9738340377807617 Train_acc: 3.250000%
Train Epoch: 3/50 Train_Loss: 3.9070000648498535 Train_acc: 3.125000%
Train Epoch: 4/50 Train_Loss: 3.893277883529663 Train_acc: 3.187500%
Train Epoch: 5/50 Train_Loss: 3.8508388996124268 Train_acc: 4.562500%
Train Epoch: 6/50 Train_Loss: 3.81877064704895 Train_acc: 6.500000%
Train Epoch: 7/50 Train_Loss: 3.7114217281341553 Train_acc: 12.312500%
Train Epoch: 8/50 Train_Loss: 3.5740270614624023 Train_acc: 22.750000%
Train Epoch: 9/50 Train_Loss: 3.3765788078308105 Train_acc: 42.562500%
Train Epoch: 10/50 Train_Loss: 3.104872703552246 Train_acc: 65.062500%
Train Epoch: 11/50 Train_Loss: 2.8033406734466553 Train_acc: 80.312500%
Train Epoch: 12/50 Train_Loss: 2.327721118927002 Train_acc: 87.312500%
Train Epoch: 13/50 Train_Loss: 2.083977460861206 Train_acc: 90.562500%
Train Epoch: 14/50 Train_Loss: 1.5242999792099 Train_acc: 91.875000%
Train Epoch: 15/5

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()